In [18]:
import pandas as pd
import numpy as np
import nltk
import os
import nltk.corpus

In [19]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')
#stop

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Komma.Devendrakumar\AppData\Roaming\nltk_data
[nltk_data]     ...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
df=pd.read_csv("C:/Users/Komma.Devendrakumar/Desktop/Cognitive-Project-master/latest_ticket_data.csv")
df.head()

,description,type,number
0,hi since recruiter lead permission approve req...,Application,1
1,re expire days hi ask help update passwords co...,Application,2
2,verification warning hi has got attached pleas...,Application,3
3,please dear looks blacklisted receiving mails ...,Application,4
4,dear modules report report cost thank much reg...,Application,5


In [21]:
from nltk.stem import SnowballStemmer, WordNetLemmatizer
snowball_stemmer = SnowballStemmer("english")
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [snowball_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

df['description'] = df['description'].apply(stem_sentences)

In [22]:
df.head()

,description,type,number
0,hi sinc recruit lead permiss approv requisit m...,Application,1
1,re expir day hi ask help updat password collea...,Application,2
2,verif warn hi has got attach pleas address bes...,Application,3
3,pleas dear look blacklist receiv mail anymor s...,Application,4
4,dear modul report report cost thank much regard,Application,5


In [23]:
def stop_words(sentence):
    tokens = sentence.split() 
    ss =  [item for item in tokens if not item in stop]
    return ' '.join(ss)
df['description'] = df['description'].apply(stop_words)

In [24]:
df.head()

,description,type,number
0,hi sinc recruit lead permiss approv requisit m...,Application,1
1,expir day hi ask help updat password colleagu ...,Application,2
2,verif warn hi got attach pleas address best re...,Application,3
3,pleas dear look blacklist receiv mail anymor s...,Application,4
4,dear modul report report cost thank much regard,Application,5


In [25]:
df['type'].value_counts()

Database            600
Security            600
Network             600
Application         600
User Maintenance    600
Name: type, dtype: int64

In [26]:
X=df['description']
y=df['type']

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30,random_state=0)
X_train.shape

(2100,)

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

In [29]:
X_train_counts.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [30]:
X_train_counts.shape

(2100, 2807)

In [31]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2100, 2807)

In [32]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [33]:
clf

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [54]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', MultinomialNB()),])
text_clf = text_clf.fit(X_train, y_train)

In [55]:
import numpy as np
predicted = text_clf.predict(X_test)
text_clf.score(X_test,y_test)

0.7388888888888889

In [52]:
predicted

array(['Network', 'Network', 'User Maintenance', 'Network', 'Database',
       'Security', 'Security', 'Database', 'User Maintenance', 'Network',
       'User Maintenance', 'User Maintenance', 'User Maintenance',
       'Application', 'User Maintenance', 'Security', 'Network',
       'Database', 'Application', 'Network', 'Security', 'Security',
       'Network', 'User Maintenance', 'Network', 'Application', 'Network',
       'Network', 'Network', 'Application', 'Network', 'User Maintenance',
       'Security', 'Database', 'User Maintenance', 'Network',
       'Application', 'User Maintenance', 'Security', 'User Maintenance',
       'Application', 'Network', 'Network', 'Application', 'Application',
       'Application', 'Database', 'Network', 'Network', 'Application',
       'Application', 'User Maintenance', 'Database', 'Security',
       'User Maintenance', 'Database', 'Security', 'Application',
       'Network', 'Network', 'Security', 'Database', 'Network',
       'User Maintenance',

In [83]:
from sklearn.externals import joblib
joblib.dump(text_clf, 'model.pkl')

def stemSentence(sentence):
    token_words = sentence.split()
    token_words
    stem_sentence = []
    for word in token_words:
        stem_sentence.append(snowball_stemmer.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

X_test1 = [stemSentence("Issue with wifi in 6th, 7th floor of UB City Building")]
    


#type(X_test1)

In [84]:
X_test1

['issu with wifi in 6th, 7th floor of ub citi build ']

In [85]:
X_test1=str(X_test1)


In [86]:
def stop_words(sentence):
    tokens = sentence.split()
    ss =  [item for item in tokens if not item in stop]
    return ' '.join(ss)


X_test1=[stop_words(X_test1)]


In [87]:
len(X_test1)

1

In [88]:
predicted = text_clf.predict(X_test1)
prob=text_clf.predict_proba(X_test1)
print(str(predicted[0]))
print(prob)
print(text_clf.classes_)
print(max(prob[0]))

Security
[[0.05141019 0.20299375 0.14569486 0.3230041  0.2768971 ]]
['Application' 'Database' 'Network' 'Security' 'User Maintenance']
0.3230041008257519
